In [35]:
import cv2
import numpy as np 
import argparse
import time
import os
import re

import matplotlib.pyplot as plt



In [1]:
#Load yolo
def load_yolo():
	net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
	classes = []
	with open("coco.names", "r") as f:
		classes = [line.strip() for line in f.readlines()]

	layers_names = net.getLayerNames()
	output_layers = [layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
	colors = np.random.uniform(0, 255, size=(len(classes), 3))
	return net, classes, colors, output_layers


In [39]:

def load_image(img_path):
	# image loading
	img = cv2.imread(img_path)
	img = cv2.resize(img, None, fx=0.4, fy=0.4)
	height, width, channels = img.shape
	return img, height, width, channels



In [41]:
def display_blob(blob):
  for b in blob:
    for n, imgb in enumerate(b):
      cv2.imshow(imgb)

In [42]:
def detect_objects(img, net, outputLayers):			
	blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
	net.setInput(blob)
	outputs = net.forward(outputLayers)
	return blob, outputs

In [43]:
def get_box_dimensions(outputs, height, width):
	boxes = []
	confs = []
	class_ids = []
	for output in outputs:
		for detect in output:
			scores = detect[5:]
			class_id = np.argmax(scores)
			conf = scores[class_id]
			if conf > 0.3 and class_id==0:
				center_x = int(detect[0] * width)
				center_y = int(detect[1] * height)
				w = int(detect[2] * width)
				h = int(detect[3] * height)
				x = int(center_x - w/2)
				y = int(center_y - h / 2)
				boxes.append([x, y, w, h])
				confs.append(float(conf))
				class_ids.append(class_id)
	return boxes, confs, class_ids

In [44]:
def draw_labels(boxes, confs, colors, class_ids, classes, img): 
  indexes=cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
  midpoint=[]
  ind=[]
  ind1=[]
  for x in indexes:
    ind1.append(x[0])
  # print(indexes)
  font=cv2.FONT_HERSHEY_PLAIN
  for i in range(len(boxes)):
    if i in indexes:
      x,y, w, h = boxes[i]
      label=str(classes[class_ids[i]])
      color=colors[i]
      cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 2)
      ind.append(i)

      #compute bottom center of bbox
      x_mid = int((x+(x+w))/2)
      y_mid = int(y+h)
      mid   = (x_mid,y_mid)
      midpoint.append(mid)
      _ = cv2.circle(img, mid, 5, (0, 0, 255), -1)

      cv2.putText(img,str(i),mid, font, 1, (255, 255, 255), 2,cv2.LINE_AA)
  return midpoint,img,(ind)

In [46]:
# %%time
from scipy.spatial import distance
def compute_distance(midpoints,ind):
  num=len(ind)

  dist = np.zeros((num,num))
  for i in range(num):
    for j in range(i+1,num):
      if  i!=j:
        dst = distance.euclidean(midpoints[i], midpoints[j])
        # print(ind[j])
        dist[i][j]=dst
  # print("dist")
  # print(dist)      
  return dist

In [47]:
%%time
def find_closest(dist,ind,thresh):
  p1=[]
  p2=[]
  d=[]
  num=len(ind)
  for i in range(num):
    for j in range(i,num):
      if( (i!=j) and (dist[i][j]<=50)):
        p1.append(ind[i])
        p2.append(ind[j])
        d.append(dist[i][j])    
  return p1,p2,d

Wall time: 0 ns


In [48]:
def change_2_red(img,person,indexes,p1,p2):
  risky = np.unique(p1+p2)
  for i in risky:
    if(i in indexes):
      x,y,w,h = person[i]
      _ = cv2.rectangle(img, (x, y), (x+w, y+h), (255,0,0), 2) 
         
  return img

In [49]:
def image_detect(img_path): 
	model, classes, colors, output_layers = load_yolo()
	image, height, width, channels = load_image(img_path)
	blob, outputs = detect_objects(image, model, output_layers)
	boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
	midpoints,img,indexes=draw_labels(boxes, confs, colors, class_ids, classes, image)
	dist=compute_distance(midpoints,indexes)
	p1,p2,d=find_closest(dist,indexes,100)
	df = pd.DataFrame({"p1":p1,"p2":p2,"dist":d})
	img=change_2_red(img,boxes,indexes,p1,p2)

In [50]:
def start_video(name):
	image = cv2.imread('frames/'+name)
	model, classes, colors, output_layers = load_yolo()
	image, height, width, channels = load_image('frames/'+name)
	blob, outputs = detect_objects(image, model, output_layers)
	boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
	midpoints,img,indexes=draw_labels(boxes, confs, colors, class_ids, classes, image)
	dist=compute_distance(midpoints,indexes)
	p1,p2,d=find_closest(dist,indexes,50)
	img=change_2_red(img,boxes,indexes,p1,p2)
	cv2.imwrite('frames/'+name,img)

In [51]:
names=os.listdir('frames/')
names.sort(key=lambda f: int(re.sub('\D', '', f)))
names

['0.png',
 '1.png',
 '2.png',
 '3.png',
 '4.png',
 '5.png',
 '6.png',
 '7.png',
 '8.png',
 '9.png',
 '10.png',
 '11.png',
 '12.png',
 '13.png',
 '14.png',
 '15.png',
 '16.png',
 '17.png',
 '18.png',
 '19.png',
 '20.png',
 '21.png',
 '22.png',
 '23.png',
 '24.png',
 '25.png',
 '26.png',
 '27.png',
 '28.png',
 '29.png',
 '30.png',
 '31.png',
 '32.png',
 '33.png',
 '34.png',
 '35.png',
 '36.png',
 '37.png',
 '38.png',
 '39.png',
 '40.png',
 '41.png',
 '42.png',
 '43.png',
 '44.png',
 '45.png',
 '46.png',
 '47.png',
 '48.png',
 '49.png',
 '50.png',
 '51.png',
 '52.png',
 '53.png',
 '54.png',
 '55.png',
 '56.png',
 '57.png',
 '58.png',
 '59.png',
 '60.png',
 '61.png',
 '62.png',
 '63.png',
 '64.png',
 '65.png',
 '66.png',
 '67.png',
 '68.png',
 '69.png',
 '70.png',
 '71.png',
 '72.png',
 '73.png',
 '74.png',
 '75.png',
 '76.png',
 '77.png',
 '78.png',
 '79.png',
 '80.png',
 '81.png',
 '82.png',
 '83.png',
 '84.png',
 '85.png',
 '86.png',
 '87.png',
 '88.png',
 '89.png',
 '90.png',
 '91.png'

In [52]:
from tqdm import tqdm
thresh=50
_ = [start_video(names[i]) for i in tqdm(range(len(names))) ]

100%|████████████████████████████████████████████████████████████████████████████████| 750/750 [17:50<00:00,  1.43s/it]


In [53]:
%%time
frames = os.listdir('frames/')
frames.sort(key=lambda f: int(re.sub('\D', '', f)))

frame_array=[]

for i in range(len(frames)):
    
    #reading each files
    img = cv2.imread('frames/'+frames[i])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)

out = cv2.VideoWriter('sample_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
 
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

Wall time: 10.5 s
